In [1]:
#导入相关的库
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from dateutil.parser import parse

In [2]:
#数据库连接初始化
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)
print(mydb)

In [3]:
# #数据库连接初始化
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="qaz!@wsx",
#     database="work"
# )
# print(mydb)

In [4]:
#定义SQL语句
profitCheck='SELECT * FROM profit_check_sku_shop'
returnDetail ='SELECT * FROM return_sku_shop'

In [5]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(profitCheck)
profitCheck1901=mycursor.fetchall()
mycursor.close()

True

In [6]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(returnDetail)
returnSku1901=mycursor.fetchall()
mycursor.close()

True

In [7]:
#数据格式转化成为pandas
profitCheck1901=pd.DataFrame(profitCheck1901)
returnSku1901=pd.DataFrame(returnSku1901)

In [8]:
profitCheck1901.columns=['Date','GoodsNO','BarCode','GoodsName','GoodsCount','Income','Cost','FavourableTotal'\
                        ,'Postage','PostageTotal','OperationCost','shopname']
returnSku1901.columns=['Date','GoodsNO','GoodsName','ReturnTotal','GoodsCount','returnCost','shopname']

In [9]:
returnSku1901.head()

,Date,GoodsNO,GoodsName,ReturnTotal,GoodsCount,returnCost,shopname
0,2019-03,00755,KUB可优比-升级款青少年床垫定做,3748.90,4.8500,873.00000000,可优比旗舰店
1,2019-03,00807,米歌-F50豪华婴儿推车（马卡龙蜜粉蓝）,2018.10,5.0000,2401.00000000,欣懿宝贝-米歌
2,2019-03,0097,KUB可优比-披风,89.00,1.0000,60.00000000,可优比旗舰店
3,2019-03,01034,蒂爱-竹纤维尿布,56.80,5.0000,15.70000000,蒂爱旗舰店
4,2019-03,01051,KUB可优比-梦维斯2015四季款婴儿床垫110*60,261.00,1.0000,112.90000000,京东可优比母婴旗舰店


In [10]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
result2=returnSku1901.groupby(['Date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result2.GoodsCount.sum()

37524.97

In [11]:
profitCheck1901.head()

,Date,GoodsNO,BarCode,GoodsName,GoodsCount,Income,Cost,FavourableTotal,Postage,PostageTotal,OperationCost,shopname
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,15648.00,可优比旗舰店
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,5757.00,可优比旗舰店
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,5010.00,可优比旗舰店
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,17086.81,可优比旗舰店
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,5624.90,可优比旗舰店


In [12]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)

In [13]:
#合并出库明细和退货明细
data_merge = pd.merge(profitCheck1901,returnSku1901,on=['Date','GoodsNO','shopname'],how='left')

In [14]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
data_merge1=data_merge.groupby(['Date'],as_index=False)
data_merge1=data_merge1.aggregate(np.sum)#出库总数
data_merge1.GoodsCount_y.sum()

37189.97

In [15]:
# data_merge.to_excel("data_merge_right.xlsx",header=True)

In [16]:
data_merge.head()

,Date,GoodsNO,BarCode,GoodsName_x,GoodsCount_x,Income,Cost,FavourableTotal,Postage,PostageTotal,OperationCost,shopname,GoodsName_y,ReturnTotal,GoodsCount_y,returnCost
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,15648.00,可优比旗舰店,KUB可优比-欢乐动物滑板车-活绿猫头鹰,14358.80,74.0,7626.44000000
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,5757.00,可优比旗舰店,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,26103.35,31.0,15940.20000000
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,5010.00,可优比旗舰店,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,44386.72,53.0,27252.60000000
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,17086.81,可优比旗舰店,KUB可优比-干湿棉柔巾,5357.32,588.0,2780.06400000
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,5624.90,可优比旗舰店,KUB可优比-铂金装加厚柔湿巾12包/箱,464.54,3.0,259.20000000


In [17]:
#data_merge.to_excel("data_merge.xlsx",header=True,index=False)

In [18]:
data_merge.head()

,Date,GoodsNO,BarCode,GoodsName_x,GoodsCount_x,Income,Cost,FavourableTotal,Postage,PostageTotal,OperationCost,shopname,GoodsName_y,ReturnTotal,GoodsCount_y,returnCost
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,15648.00,可优比旗舰店,KUB可优比-欢乐动物滑板车-活绿猫头鹰,14358.80,74.0,7626.44000000
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,5757.00,可优比旗舰店,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,26103.35,31.0,15940.20000000
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,5010.00,可优比旗舰店,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,44386.72,53.0,27252.60000000
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,17086.81,可优比旗舰店,KUB可优比-干湿棉柔巾,5357.32,588.0,2780.06400000
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,5624.90,可优比旗舰店,KUB可优比-铂金装加厚柔湿巾12包/箱,464.54,3.0,259.20000000


In [19]:
del data_merge['GoodsName_y']

In [20]:
#获取商品小组对应情况
GoodsInfo =pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\goodsinfo0305.csv',header = 0,engine='python')
GoodsInfo.columns=['GoodsNO','GoodsName','BarCode','Company','Dept','TmallTax','GroupName','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [21]:
#删除重复以及不必要列
GoodsInfoShort=GoodsInfo[['Company','Dept','GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU','BarCode']]

In [22]:
GoodsInfoShort.shape

(17337, 10)

In [23]:
#以SKU编码为唯一辨识，去重
GoodsInfoShort=GoodsInfoShort.drop_duplicates(["BarCode"])

In [24]:
GoodsInfoShort.shape

(17117, 10)

In [25]:
GoodsInfoShort.head()

,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU,BarCode
0,可优比母婴用品有限公司,KUB穿戴,礼盒配饰组,0.05,谢涛涛,婴童配饰,婴童饰品,领结,K050231,6971971884682
1,可优比母婴用品有限公司,KUB穿戴,礼盒配饰组,0.05,谢涛涛,婴童配饰,婴童饰品,领结,K050231,6971971884699
2,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209,6971971884040
3,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209,6971971884033
4,可优比母婴用品有限公司,KUB穿戴,内服组,0.05,王江峰,婴童外衣,外套/夹克/大衣,其它外套,K050209,6971971884071


In [26]:
#匹配销售数据和对应小组信息
data_merge_dept=pd.merge(data_merge,GoodsInfoShort,on=['BarCode'],how='left')

In [27]:
# data_merge_dept = pd.read_csv('C:\\Users\\Administrator\\Desktop\\sell_return_dept.csv')

In [28]:
data_merge_dept.head()

,Date,GoodsNO,BarCode,GoodsName_x,GoodsCount_x,Income,Cost,FavourableTotal,Postage,PostageTotal,...,returnCost,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,...,7626.44000000,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,户外运动,儿童车类,滑板车,WJ0406
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,...,15940.20000000,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8511
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,...,27252.60000000,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,...,2780.06400000,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP047
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,...,259.20000000,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP


In [29]:
data_merge_dept.columns

Index(['Date', 'GoodsNO', 'BarCode', 'GoodsName_x', 'GoodsCount_x', 'Income',
       'Cost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'shopname', 'ReturnTotal', 'GoodsCount_y', 'returnCost', 'Company',
       'Dept', 'GroupName', 'TmallTax', 'Holder', 'GradeTwo', 'GradeThird',
       'GradeForth', 'SPU'],
      dtype='object')

In [30]:
# kub=pd.read_csv('C:\\Users\\Administrator\\Desktop\\商品SKU事业部\\0124\\kub0125.csv',engine='python')
# kub=kub[['GoodsNO','Dept','GroupName']]
# kub=kub.drop_duplicates(["GoodsNO"])
# kub.columns=['SKU','Dept','GroupName']
# data_merge_dept_gp=pd.merge(data_merge_dept,kub,on=['SKU'],how='left')
# data_merge_dept_gp.to_excel('data_group_new0125.xlsx',header=True,index=False)

In [31]:
#修改合并后的列名
data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost','ShopName',
       'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
       'GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [32]:
data_merge_dept.head()

,Date,SKU,BarCode,Name,SellCount,SellIncome,SellCost,FavourableTotal,Postage,PostageTotal,...,ReturnCost,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,...,7626.44000000,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,户外运动,儿童车类,滑板车,WJ0406
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,...,15940.20000000,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8511
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,...,27252.60000000,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,...,2780.06400000,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP047
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,...,259.20000000,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP


In [33]:
shopname={'贝豪婴童专营店':'天猫商城',
'欣懿宝贝-米歌':'天猫商城',
'可优比旗舰店':'天猫商城',
'蒂爱旗舰店':'天猫商城',
'贝豪拼多多':'拼多多',
'麦可适旗舰店':'天猫商城',
'可优比（杭州来福士）':'线下',
'唯品会（蒂爱）':'唯品会',
'美易仓库':'美易仓库',
'京东可优比母婴旗舰店':'京东商城',
'可优比海外自营店':'海外店',
'可优比天猫超市':'天猫超市',
'唯品会（可优比）':'唯品会',
'唯品会MP（蒂爱）':'唯品会',
'优趣玩具专营店':'天猫商城',
'可优比母婴生活馆':'有赞',
'贝豪贝店商城':'贝贝网',
'可优比团购店铺':'团购',
'蒂爱团购店铺':'团购',
'京东蒂爱母婴旗舰店':'京东商城'}

In [34]:
data_merge_dept['Source']=data_merge_dept['ShopName'].map(shopname)

In [35]:
#data_merge_dept['Postage']=0

In [36]:
data_merge_dept['GroupName']=data_merge_dept['GroupName'].fillna('#N/A')

In [37]:
data_merge_dept=data_merge_dept.drop_duplicates(["SKU","ShopName"])

In [38]:
data_merge_dept.to_csv("C:\\Users\\Administrator\\Desktop\\data_merge_dept.csv",header=True,index=False)
data_merge_dept.to_excel("C:\\Users\\Administrator\\Desktop\\data_merge_dept.xlsx",header=True,index=False)

In [39]:
department_gross_profit = pd.DataFrame()
department_gross_profit = pd.concat([department_gross_profit,data_merge_dept])

In [40]:
#查看缺失值情况
department_gross_profit=department_gross_profit.fillna(np.nan)
department_gross_profit.isnull().sum()

Date                  0
SKU                   0
BarCode               9
Name                  0
SellCount             0
SellIncome            0
SellCost              0
FavourableTotal       0
Postage               0
PostageTotal          0
OperationCost         0
ShopName              0
ReturnTotal        6182
ReturnCount        6182
ReturnCost         6182
Company             137
Dept                137
GroupName             0
TmallTax            137
Holder             1182
GradeTwo           1159
GradeThird         1159
GradeForth         1159
SPU                2174
Source               11
dtype: int64

In [41]:
department_gross_profit.head()

,Date,SKU,BarCode,Name,SellCount,SellIncome,SellCost,FavourableTotal,Postage,PostageTotal,...,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU,Source
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,...,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,户外运动,儿童车类,滑板车,WJ0406,天猫商城
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,...,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8511,天猫商城
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,...,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512,天猫商城
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,...,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP047,天猫商城
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,...,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP,天猫商城


In [42]:
department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']] = department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].astype(float)

In [43]:
department_gross_profit.head()

,Date,SKU,BarCode,Name,SellCount,SellIncome,SellCost,FavourableTotal,Postage,PostageTotal,...,Company,Dept,GroupName,TmallTax,Holder,GradeTwo,GradeThird,GradeForth,SPU,Source
0,2019-03,C.WJ.03.01.000147,6925125575544,KUB可优比-欢乐动物滑板车-活绿猫头鹰,5223.0,1409110.00,538282.38,566786.03,47012.10,583.00,...,可优比母婴用品有限公司,KUB玩教,学爬健身组,0.05,魏愿/陆昌胜,户外运动,儿童车类,滑板车,WJ0406,天猫商城
1,2019-03,K04865301,6971971889076,KUB可优比-艾迪斯婴儿床-旗舰版猪年生肖-120*65,1925.0,1715003.35,989835.00,188797.49,0.00,18993.00,...,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8511,天猫商城
2,2019-03,C.TC.01.01.000059,6925125592633,KUB可优比-艾迪斯婴儿床-新款白马FD826-120*65,1677.0,1484405.92,862313.40,172432.84,0.00,5187.00,...,可优比母婴用品有限公司,KUB睡眠,睡床小组,0.05,林嘉欣,婴童卧室家具,婴童床,实木床,SL8512,天猫商城
3,2019-03,C.XH.13.01.000008,6925125569666,KUB可优比-干湿棉柔巾,68773.0,701463.37,325158.74,147125.49,7075.87,301.04,...,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP047,天猫商城
4,2019-03,C.XH.12.01.000018,6925125595610,KUB可优比-铂金装加厚柔湿巾12包/箱,1882.0,356053.00,162604.80,126540.21,14119.80,0.00,...,可优比母婴用品有限公司,KUB用品,大快消,0.02,王琳玉,快消类,干湿巾类,手口专用巾,YP,天猫商城


In [44]:
#填充缺失值为0
department_gross_profit[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']]=department_gross_profit\
[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].fillna(0)

#填充缺失小组为#N/A
department_gross_profit['GroupName']=department_gross_profit\
['GroupName'].fillna('#N/A')

In [45]:
department_gross_profit['CheckIncome'] = department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']\
                                        +department_gross_profit['PostageTotal']-department_gross_profit['ReturnTotal']
department_gross_profit['CheckCost'] = department_gross_profit['SellCost']-department_gross_profit['ReturnCost']
department_gross_profit['TmallCharge'] = (department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']-\
                                        department_gross_profit['ReturnTotal'])*department_gross_profit['TmallTax']

In [ ]:
# #按公司，部门，日期进行归类
# department_gross_profit_gb=department_gross_profit.groupby(['Company','Dept','Date','Source','ShopName'],as_index=False)

In [46]:
#按公司，部门，日期进行归类
department_gross_profit_gb=department_gross_profit.groupby(['Company','Dept','Date'],as_index=False)

In [47]:
#归类汇总
department_gross_profit_gb = department_gross_profit_gb.aggregate(np.sum)
del department_gross_profit_gb['TmallTax']

In [48]:
#计算营业税，毛利润，毛利率
department_gross_profit_gb['Tax']=department_gross_profit_gb['CheckIncome']*0.06
department_gross_profit_gb['GrossProfit']=department_gross_profit_gb['CheckIncome']-department_gross_profit_gb['CheckCost']-\
                                            department_gross_profit_gb['CheckIncome']*0.06-department_gross_profit_gb['TmallCharge']-\
                                            department_gross_profit_gb['Postage']-department_gross_profit_gb['OperationCost']
department_gross_profit_gb['GrossProfitRate']=department_gross_profit_gb['GrossProfit']/department_gross_profit_gb['SellIncome']

In [49]:
department_gross_profit_gb=department_gross_profit_gb[(department_gross_profit_gb['Dept']!='KUB包材')&(department_gross_profit_gb['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb['Dept']!='蒂爱共用')]

In [50]:
department_gross_profit_gb.head()

,Company,Dept,Date,SellCount,SellIncome,SellCost,FavourableTotal,Postage,PostageTotal,OperationCost,ReturnTotal,ReturnCount,ReturnCost,CheckIncome,CheckCost,TmallCharge,Tax,GrossProfit,GrossProfitRate
0,可优比母婴用品有限公司,KUB出行,2019-03,11480.0,3084060.34,1599804.78,298411.52,20730.57,5264.63,33042.19,219390.11,937.0,119813.487,2571523.34,1.479991e+06,1.169090e+05,1.542914e+05,7.665589e+05,0.248555
2,可优比母婴用品有限公司,KUB喂辅,2019-03,171203.0,14804551.59,6877881.94,1502670.22,525164.71,16113.22,399929.59,624292.20,5583.0,324908.942,12693702.39,6.552973e+06,3.525295e+05,7.616221e+05,4.101483e+06,0.277042
3,可优比母婴用品有限公司,KUB孕产,2019-03,57826.0,1623114.44,771462.85,159980.62,78544.19,3626.07,64561.46,51846.94,1026.0,26721.920,1414912.95,7.447409e+05,4.966410e+04,8.489478e+04,3.925075e+05,0.241824
4,可优比母婴用品有限公司,KUB安防,2019-03,90826.0,4673457.57,1926369.58,414459.93,159822.30,10280.93,155038.44,164562.96,2140.0,81796.530,4104715.61,1.844573e+06,1.010047e+05,2.462829e+05,1.597994e+06,0.341930
5,可优比母婴用品有限公司,KUB玩教,2019-03,174510.0,26405699.05,12437345.63,2841237.95,519373.81,32968.70,425202.89,806282.20,4930.0,415581.504,22791147.60,1.202176e+07,1.107027e+06,1.367469e+06,7.350311e+06,0.278361


In [51]:
department_gross_profit_gb.to_csv("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb.csv",header=True,index=False)


In [55]:
department_gross_profit_gb.columns=['公司','事业部','日期','销售数量','销售收入','销售货品成本','优惠金额','快递费','应收邮资','操作费','退货金额','退货数量','退货货品成本','核算收入','核算成本','天猫扣点','营业税','毛利润','毛利率']
department_gross_profit_gb.to_excel("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb.xlsx",header=True,index=False)

In [ ]:
# data_merge1=department_gross_profit_gb.groupby(['Date'],as_index=False)
# data_merge1=data_merge1.aggregate(np.sum)#出库总数
# data_merge1.ReturnCount.sum()

In [ ]:
#按公司、部门、小组已经日期进行归类
department_gross_profit_gb_group=department_gross_profit.groupby(['Company','Dept','GroupName','Date','Source','ShopName'],as_index=False)

In [ ]:
department_gross_profit_gb_group = department_gross_profit_gb_group.aggregate(np.sum)
del department_gross_profit_gb_group['TmallTax']

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group['Tax']=department_gross_profit_gb_group['CheckIncome']*0.06
department_gross_profit_gb_group['GrossProfit']=department_gross_profit_gb_group['CheckIncome']-department_gross_profit_gb_group['CheckCost']-\
                                            department_gross_profit_gb_group['CheckIncome']*0.06-department_gross_profit_gb_group['TmallCharge']-\
                                            department_gross_profit_gb_group['Postage']-department_gross_profit_gb_group['OperationCost']
department_gross_profit_gb_group['GrossProfitRate']=department_gross_profit_gb_group['GrossProfit']/department_gross_profit_gb_group['SellIncome']

In [ ]:
department_gross_profit_gb_group=department_gross_profit_gb_group[(department_gross_profit_gb_group['Dept']!='KUB包材')&(department_gross_profit_gb_group['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb_group['Dept']!='蒂爱共用')]

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group.to_csv("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb_group.csv",header=True,index=False)
department_gross_profit_gb_group.to_excel("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb_group.xlsx",header=True,index=False)

In [ ]:
data_merge_dept_edit=pd.DataFrame()
data_merge_dept_edit=pd.concat([data_merge_dept_edit,data_merge_dept])

In [ ]:
data_merge_dept_edit.columns=['日期','SKU编码','条码','品名','销售数量','销售收入','货品成本','优惠金额','快递费',\
                        '应收邮资','操作费','店铺名称','退货金额','退货数量','退货货品成本','公司名称','事业部','小组','天猫扣点','负责人','二级类目','三级类目','四级类目','SPU','渠道']

In [ ]:
data_merge_dept_edit.columns

In [ ]:
#填充缺失值为0
data_merge_dept_edit=data_merge_dept_edit.fillna(np.nan)
data_merge_dept_edit.isnull().sum()

In [ ]:
# data_merge_dept_edit[['操作费','退货金额','退货数量','退货货品成本']]=data_merge_dept_edit\
# [['操作费','退货金额','退货数量','退货货品成本']].fillna(0)

In [ ]:
# data_merge_dept_edit.to_excel("20190131.xlsx",header=True,index=False)
data_merge_dept_edit.to_excel("C:\\Users\\Administrator\\Desktop\\data_merge_dept.xlsx",header=True,index=False)

In [ ]:
# data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
#        'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
#        'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
#        'Group']

In [ ]:
# def merge(data_left,data_right,key):
#     left=pd.read_csv(data_left,header = 0,dtype = object,engine='python',encoding='utf-8')
#     right=pd.read_csv(data_right,header = 0,dtype = object,engine='python')
#     right=right.drop_duplicates(key)
#     data_merge=pd.merge(left,right,on=key,how='left')
#     return data_merge

In [ ]:
# del data['BarCode_x']
# data['BarCode']=data['BarCode_y']
# data=data.drop('BarCode_y',axis=1)